In [3]:
import pandas as pd
import ccxt
from tqdm import tqdm
import time
import random


In [38]:
binance = ccxt.binance()


pairs = [pair for pair in binance.load_markets() if pair.endswith("USDT")]

print("number of pairs :", len(pairs))

#percentage = float(input("Enter the percentage of data you want to fetch: "))
percentage =5
print(f"Fetching {percentage}% of data...")
if percentage < 1 or percentage > 100:

    raise ValueError("Percentage must be between 1 and 100")

select_pairs = random.sample(pairs, int(len(pairs) * percentage / 100))

timeframes = ["1h"]
all_data = []

# Calculate the start date based on the number of candles we want
for timeframe in timeframes:
    print(f"Fetching data for {timeframe} timeframe...")
    for pair in tqdm(select_pairs):
        try:
            # Fetch last 10 candles directly
            candles = binance.fetch_ohlcv(pair, timeframe, limit=15)

            for candle in candles:
                timestamp, open_, high, low, close, volume = candle
                all_data.append(
                    {
                        "pair": pair,
                        "timeframe": timeframe,
                        "timestamp": pd.to_datetime(timestamp, unit="ms"),
                        "open": open_,
                        "high": high,
                        "low": low,
                        "close": close,
                        "volume": volume,
                    }
                )

            time.sleep(1)  # Rate limiting

        except Exception as e:
            print(f"Error fetching data for {pair} on {timeframe}: {e}")

# Convert to DataFrame with timeframe information
df = pd.DataFrame(all_data)

print(df)


number of pairs : 888
Fetching 5% of data...
Fetching data for 1h timeframe...


100%|██████████| 44/44 [00:55<00:00,  1.27s/it]

               pair timeframe           timestamp    open    high     low  \
0          OAX/USDT        1h 2024-12-09 12:00:00  0.0646  0.0650  0.0620   
1          OAX/USDT        1h 2024-12-09 13:00:00  0.0643  0.0656  0.0619   
2          OAX/USDT        1h 2024-12-09 14:00:00  0.0629  0.0630  0.0588   
3          OAX/USDT        1h 2024-12-09 15:00:00  0.0615  0.0617  0.0552   
4          OAX/USDT        1h 2024-12-09 16:00:00  0.0586  0.0617  0.0566   
..              ...       ...                 ...     ...     ...     ...   
655  FLUX/USDT:USDT        1h 2024-12-11 17:00:00  0.7562  0.7625  0.7442   
656  FLUX/USDT:USDT        1h 2024-12-11 18:00:00  0.7459  0.7579  0.7445   
657  FLUX/USDT:USDT        1h 2024-12-11 19:00:00  0.7556  0.7584  0.7513   
658  FLUX/USDT:USDT        1h 2024-12-11 20:00:00  0.7569  0.7625  0.7511   
659  FLUX/USDT:USDT        1h 2024-12-11 21:00:00  0.7533  0.7541  0.7526   

      close     volume  
0    0.0643  1770626.0  
1    0.0629  1317579.0  


In [48]:

pairs = df["pair"].unique()


def buy_or_sell_candell(id, df) -> bool:
    return df.loc[id]["close"] > df.loc[id]["open"]

print(pairs)

['OAX/USDT' 'HIGH/USDT' 'XTZUP/USDT' 'TAO/USDT' 'CTSI/USDT:USDT' 'HC/USDT'
 'DYM/USDT' 'ALICE/USDT:USDT' 'STORM/USDT' 'RUNE/USDT' 'PIXEL/USDT:USDT'
 'GALA/USDT:USDT' 'POPCAT/USDT:USDT' 'XTZDOWN/USDT' 'SCR/USDT'
 'FIO/USDT:USDT' 'BURGER/USDT' 'IDEX/USDT:USDT' 'BNBBULL/USDT' 'MOB/USDT'
 'HARD/USDT' 'VET/USDT:USDT' 'EOSDOWN/USDT' 'WBETH/USDT' 'CHESS/USDT'
 'GALA/USDT' 'CTSI/USDT' 'GTO/USDT' 'AR/USDT' 'MASK/USDT'
 'AUCTION/USDT:USDT' 'GMX/USDT:USDT' 'WAVES/USDT:USDT' 'UNI/USDT:USDT'
 'XEC/USDT' 'ATOM/USDT' 'ORCA/USDT' 'THETA/USDT:USDT' 'WOO/USDT'
 'DOT/USDT' 'JTO/USDT' '1000SHIB/USDT:USDT' 'VITE/USDT' 'FLUX/USDT:USDT']


In [ ]:
for pair in pairs[:4]:
    for timeframe in timeframes:
        df_pair_timeframe = df[
            (df["pair"] == pair) & (df["timeframe"] == timeframe)
        ].copy()
        
        if df_pair_timeframe.empty:
            print(f"No data for pair {pair} and timeframe {timeframe}")
            continue
        
        df_pair_timeframe = df_pair_timeframe.reset_index(drop=True)
        
        #print(df_pair_timeframe)
        
        # Avoid division by zero
        if (df_pair_timeframe["low"] == 0).any():
            print(f"Zero low value found for pair {pair} and timeframe {timeframe}")
            continue
        
        df_pair_timeframe["high_low_range"] = (
            (df_pair_timeframe["high"] - df_pair_timeframe["low"])
            * 100
            / df_pair_timeframe["low"]
        )
        print(df_pair_timeframe['high_low_range'])
        
        # Check if high_low_range has valid data
        if df_pair_timeframe["high_low_range"].empty:
            print(f"No high_low_range data for pair {pair} and timeframe {timeframe}")
            continue
        
        bigger_candel = df_pair_timeframe["high_low_range"].idxmax()
        
        print(f"Bigger candle id: {bigger_candel}")
        
        # Determines whether the "bigger candle" is a green (buy) or red (sell) candle.
        
        # Args:
        #     bigger_candel (int): The index of the "bigger candle" in the DataFrame.
        #     df_pair_timeframe (pandas.DataFrame): The DataFrame containing the price data for the current pair and timeframe.
        
        # Returns:
        #     bool: True if the "bigger candle" is a green (buy) candle, False otherwise.
        # green_big_candel = buy_or_sell_candell(bigger_candel, df_pair_timeframe)
        
        # if not green_big_candel:
        #     continue
        

        # next_candels = df_pair_timeframe.loc[bigger_candel + 1 :]
        
        # if next_candels.empty:
        #     #print(f"No candles after index {bigger_candel} for pair {pair}")
        #     continue
        
        # price_high = df_pair_timeframe.loc[bigger_candel, "high"]
        # price_low = df_pair_timeframe.loc[bigger_candel, "low"]
        # price_50 = (price_high + price_low) / 2
        # price_3 = (price_high + price_low) * 0.97
        # price_5 = (price_high + price_low) * 0.95

        # exceeds_high = any(
        #     candle_high > price_high for candle_high in next_candels["high"]
        # )
        # below_50 = any(candle_low < price_50 for candle_low in next_candels["low"])

        # last_candle_high = next_candels.tail(1)["high"].values[0]

        # between_5_0 = price_5 <= last_candle_high <= price_high
        
        # price_30 = (price_high + price_low) * 0.7
        
        # between_30_0 = price_30 <= last_candle_high <= price_high

        # if (
        #     not exceeds_high
        #     and not below_50
        #     #and between_5_0
        #     #and between_30_0
        # ):
        #     print(
        #         f"Pair: {pair} | Timeframe: {timeframe} | Price high: {price_high} | Number of candles: {len(next_candels)}"
        #     )


0      4.838710
1      5.977383
2      7.142857
3     11.775362
4      9.010601
5     11.921708
6      7.233273
7      8.909091
8     29.571106
9     34.146341
10    10.859729
11     8.478261
12    10.000000
13    17.831325
14    21.348315
Name: high_low_range, dtype: float64
Bigger candle id: 9
0     1.892915
1     3.023758
2     5.193370
3     2.152853
4     1.548318
5     2.019129
6     3.511530
7     4.500776
8     2.414487
9     1.890547
10    2.713568
11    1.557007
12    1.398601
13    1.553885
14    0.100000
Name: high_low_range, dtype: float64
Bigger candle id: 2
0      3.005554
1      8.000000
2      5.015385
3      6.206897
4      3.867574
5      4.056984
6     11.585166
7     16.676923
8     11.111111
9     15.805243
10     9.644860
11     6.188467
12     7.692308
13    16.923077
14     5.573506
Name: high_low_range, dtype: float64
Bigger candle id: 13
